In [26]:
import os
import json
from datetime import datetime
import pandas as pd
from sqlalchemy import text
# from socceraction.spadl import config as spadlconfig

from utils.logger import create_logger
from utils.db_utils import calculate_statistics, get_lineup


# logger = create_logger()

class DataBase:  
    def __init__(self, engine):  
        self.engine = engine

    def init_db(self) -> None:
        """ initialization of tables in PostgresDB """
        # try:
        connection = self.engine.raw_connection()
        cursor = connection.cursor()
        cursor.execute("DROP TABLE IF EXISTS spadl;")
        cursor.execute("DROP TABLE IF EXISTS dashboard;")
        connection.commit()
        cursor.close()
        connection.close()

        with self.engine.connect() as connection:
            connection.execute(text("""
                    CREATE TABLE spadl (
                        game_id INTEGER PRIMARY KEY,
                        original_event_id INTEGER NOT NULL
                    );
            """))

            connection.execute(text("""
                    CREATE TABLE dashboard (
                        game_id TEXT NOT NULL,
                        original_event_id INTEGER NOT NULL,
                        team_id INTEGER NOT NULL,
                        player_id INTEGER NOT NULL,
                        player TEXT NOT NULL,
                        team TEXT NOT NULL,
                        pass INTEGER NOT NULL,
                        crosses INTEGER NOT NULL,
                        throw_in INTEGER NOT NULL,
                        freekick_crossed INTEGER NOT NULL,
                        freekick_short INTEGER NOT NULL,
                        corner_crossed INTEGER NOT NULL,
                        corner_short INTEGER NOT NULL,
                        take_on INTEGER NOT NULL,
                        foul INTEGER NOT NULL,
                        tackle INTEGER NOT NULL,
                        interception INTEGER NOT NULL,
                        shot INTEGER NOT NULL,
                        shot_penalty INTEGER NOT NULL,
                        shot_freekick INTEGER NOT NULL,
                        keeper_save INTEGER NOT NULL,
                        keeper_claim INTEGER NOT NULL,
                        keeper_punch INTEGER NOT NULL,
                        keeper_pick_up INTEGER NOT NULL,
                        clearance INTEGER NOT NULL,
                        bad_touch INTEGER NOT NULL,
                        non_action INTEGER NOT NULL,
                        dribble INTEGER NOT NULL,
                        goalkick INTEGER NOT NULL,
                        timestamp TIMESTAMP NOT NULL,
                        PRIMARY KEY (game_id, player_id)
                    );
            """))

        # except:
        #     logger.error("Tables in database were not initializated")
    
    def save_spadl(self, data: pd.DataFrame) -> None:

        columns = ['game_id', 'original_event_id']
        
        try:
            data[columns].to_sql("spadl", self.engine, if_exists="append", index=False)
        except:
            print("Error with writing in spadl table")


In [2]:
import json
import pandas as pd
from pathlib import Path
import soccerdata as sd
from tqdm import tqdm

from utils.logger import create_logger
from utils.db_utils import get_connection
from db.database import DataBase

# logger = create_logger()

LEAGUES = [
    'ENG-Premier League',
 ]

SEASONS = [
    '18-19',
]

def save_data(db:DataBase) -> None:
    """ Saving data from files in database db"""

    for league in LEAGUES:
        for season in SEASONS:
            try:
                ws = sd.WhoScored(leagues=league, seasons=season, no_cache=False, no_store=False)
                schedule = ws.read_schedule(force_cache=True)
            except:
                continue

            for match_id in tqdm(schedule.game_id.values):
                spadl_df = ws.read_events(match_id=int(match_id), output_fmt="spadl", force_cache=True)

                # logger.info(f"{len(spadl_df)}")  

                if len(spadl_df) == 0 or 'type_id' not in spadl_df.columns:
                    continue
                
                
                db.save_spadl(spadl_df)
    
    # logger.info(f"Spadl data was saved in database")
    
    # db.save_dashboard()
    # logger.info(f"Dashboard was saved in database")


/Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[04/04/25 14:47:27] INFO     No custom team name replacements found. You can configure these in       ]8;id=602144;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=287945;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_config.py#91\91]8;;\
                             /Users/vitaly/soccerdata/config/teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=161056;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=571859;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_config.py#197\197]8;;\
                             /Users/vitaly/soccerdata/config/league_dict.json.                                     

In [27]:
from dotenv import load_dotenv

In [28]:
load_dotenv()

True

In [30]:
from utils.db_utils import get_connection

In [31]:
engine = get_connection()

In [33]:
db = DataBase(engine)
# db.init_db()

In [ ]:
with engine.connect() as connection:
    connection.execute(text("""
            CREATE TABLE spadl (
                game_id INTEGER PRIMARY KEY,
                original_event_id INTEGER NOT NULL
            );
    """))

AttributeError: __enter__

In [49]:
query = """SELECT * FROM users"""
df = pd.read_sql(query, db.engine)
df

,id,name,age


In [52]:
query = """SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'"""
df = pd.read_sql_query(query, db.engine)
df

,table_name
0,spadl
1,dashboard


In [51]:
query = """DROP TABLE "Educative_Answers" cascade; """
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [35]:
query = """CREATE TABLE users (
  id SERIAL PRIMARY KEY,
  name VARCHAR(255) NOT NULL,
  email VARCHAR(255) NOT NULL,
  age INT NOT NULL
);
"""
df = pd.read_sql_query(query, db.engine)
df

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [ ]:
def main():
    # Database initialization
    engine = get_connection()
    db = DataBase(engine)
    db.init_db()

    logger.info("Tables in database were initializated")

    # query = f"""SELECT *
    #             FROM spadl;"""
    # df = pd.read_sql(query, db.engine)
    # # df = pd.read_sql_table('spadl', db.engine)
    # print(df)

    # Saving files into DB
    # save_data(db)
    # logger.info("Create DB was finished") 

if __name__ == "__main__":
    main()

In [23]:
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base

# Define Table Classes
Base = declarative_base()

[04/04/25 10:39:29] WARNING  /var/folders/g6/0lc4wg1n697frk1jltm6441c0000gn/T/ipykernel_17007/39141 ]8;id=759387;file:///Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/warnings.py\warnings.py]8;;\:]8;id=297572;file:///Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/warnings.py#109\109]8;;\
                             91382.py:5: MovedIn20Warning: The ``declarative_base()`` function is                  
                             now available as sqlalchemy.orm.declarative_base(). (deprecated since:                
                             2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)                    
                               Base = declarative_base()                                                           
                                                                                                                   

In [24]:
class CustomTable1(Base):
    __tablename__ = 'Educative_Answers'

    custom_id = Column(Integer, primary_key=True)
    custom_name = Column(String, unique=True)

In [25]:
# Replace 'your_database_url_here' with your actual database URL
engine = get_connection()

# Create the tables in the database
Base.metadata.create_all(engine)

In [54]:
# Create the tables in the in-memory database
Base.metadata.create_all(engine)

# Print the names of all tables in the database
def print_all_tables(engine):
    # To load metdata and existing database schema
    metadata = MetaData()
    metadata.reflect(bind=engine)
    
    tables = metadata.tables.keys()
    
    print("List of tables:")
    for table in tables:
        print(table)

# Print all tables in the in-memory database
print_all_tables(engine)

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s]
[parameters: {'table_name': 'Educative_Answers', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [52]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

# Set up the database connection
engine = get_connection()
metadata = MetaData()

# Define a table using SQLAlchemy
users_table = Table(
    "users", metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("name", String),
    Column("age", Integer),
)

users_table2 = Table(
    "users2", metadata,
    Column("id", Integer, primary_key=True, autoincrement=True),
    Column("name", String),
    Column("age", Integer),
)

# Create the table in the database
metadata.create_all(engine)

# Create a sample DataFrame
df = pd.DataFrame({
    "name": ["Alice", "Bob"],
    "age": [30, 25]
})

# Write the DataFrame to the SQL table
# df.to_sql("users", engine, if_exists="append", index=False)

# Read it back into a new DataFrame
df_from_db = pd.read_sql("SELECT * FROM users", engine)

print(df_from_db)


Empty DataFrame
Columns: [id, name, age]
Index: []


In [46]:
users_table = Table("users", metadata, autoload_with=engine)
users_table.drop(engine)

In [ ]:
import json
import pandas as pd
from pathlib import Path
import soccerdata as sd
from tqdm import tqdm

from utils.logger import create_logger
from utils.db_utils import get_connection
from db.database import DataBase

LEAGUES = [
    'GER-Bundesliga',
 ]

SEASONS = [
    '21-22',
]

def save_data() -> None:
    """ Saving data from files in database db"""

    for league in LEAGUES:
        for season in SEASONS:
            ws = sd.WhoScored(leagues=league, seasons=season, no_cache=False, no_store=False, path_to_browser=)
            schedule = ws.read_schedule(force_cache=True)

            for match_id in tqdm(schedule.game_id.values):
                spadl_df = ws.read_events(match_id=int(match_id), output_fmt="spadl", force_cache=True)

                print(f"spadl len: {len(spadl_df)}")  

                if len(spadl_df) == 0 or 'type_id' not in spadl_df.columns:
                    continue
    
    print(f"Spadl data was saved in database")

In [15]:
# save_data()

In [1]:
import os
os.environ["SOCCERDATA_LOGLEVEL"] = 'ERROR'

In [15]:
from soccerdata._common import BaseReader, BaseRequestsReader

In [16]:
from soccerdata._config import DATA_DIR, LEAGUE_DICT, MAXAGE, TEAMNAME_REPLACEMENTS, logger

In [17]:
1

1

In [19]:
import re
import json
import pandas as pd
import numpy as np
from pathlib import Path
import itertools
from lxml import html
from typing import Callable, Literal, Optional, Union, IO
from collections.abc import Iterable

WHOSCORED_DATADIR = Path('/Users/vitaly/soccerdata/data/WhoScored')

COLS_EVENTS = {
    # The ID of the game
    "game_id": np.nan,
    # 'PreMatch', 'FirstHalf', 'SecondHalf', 'PostGame'
    "period": np.nan,
    # Integer indicating the minute of the event, ignoring stoppage time
    "minute": -1,
    # Integer indicating the second of the event, ignoring stoppage time
    "second": -1,
    # Integer indicating the minute of the event, taking into account stoppage time
    "expanded_minute": -1,
    # String describing the event type (e.g. 'Goal', 'Yellow Card', etc.)
    "type": np.nan,
    # String describing the event outcome ('Succesful' or 'Unsuccessful')
    "outcome_type": np.nan,
    # The ID of the team that the event is associated with
    "team_id": np.nan,
    # The name of the team that the event is associated with
    "team": np.nan,
    # The ID of the player that the event is associated with
    "player_id": np.nan,
    # The name of the player that the event is associated with
    "player": np.nan,
    # Coordinates of the event's location
    "x": np.nan,
    "y": np.nan,
    "end_x": np.nan,
    "end_y": np.nan,
    # Coordinates of a shot's location
    "goal_mouth_y": np.nan,
    "goal_mouth_z": np.nan,
    # The coordinates where the ball was blocked
    "blocked_x": np.nan,
    "blocked_y": np.nan,
    # List of dicts with event qualifiers
    "qualifiers": [],
    # Some boolean flags
    "is_touch": False,
    "is_shot": False,
    "is_goal": False,
    # 'Yellow', 'Red', 'SecondYellow'
    "card_type": np.nan,
    # The ID of an associated event
    "related_event_id": np.nan,
    # The ID of a secondary player that the event is associated with
    "related_player_id": np.nan,
}

def _parse_url(url: str) -> dict:
    """Parse a URL from WhoScored.

    Parameters
    ----------
    url : str
        URL to parse.

    Raises
    ------
    ValueError
        If the URL could not be parsed.

    Returns
    -------
    dict
    """
    patt = (
        r"^(?:https:\/\/www.whoscored.com)?\/"
        + r"(?:regions\/(\d+)\/)?"
        + r"(?:tournaments\/(\d+)\/)?"
        + r"(?:seasons\/(\d+)\/)?"
        + r"(?:stages\/(\d+)\/)?"
        + r"(?:matches\/(\d+)\/)?"
    )
    matches = re.search(patt, url, re.IGNORECASE)
    if matches:
        return {
            "region_id": matches.group(1),
            "league_id": matches.group(2),
            "season_id": matches.group(3),
            "stage_id": matches.group(4),
            "match_id": matches.group(5),
        }
    raise ValueError(f"Could not parse URL: {url}")


def make_game_id(row: pd.Series) -> str:
    """Return a game id based on date, home and away team."""
    if pd.isnull(row["date"]):
        game_id = "{}-{}".format(
            row["home_team"],
            row["away_team"],
        )
    else:
        game_id = "{} {}-{}".format(
            row["date"].strftime("%Y-%m-%d"),
            row["home_team"],
            row["away_team"],
        )
    return game_id

def standardize_colnames(df: pd.DataFrame, cols: Optional[list[str]] = None) -> pd.DataFrame:
    """Convert DataFrame column names to snake case."""

    def to_snake(name: str) -> str:
        name = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
        name = re.sub("__([A-Z])", r"_\1", name)
        name = re.sub("([a-z0-9])([A-Z])", r"\1_\2", name)
        return name.lower().replace("-", "_").replace(" ", "")

    if df.columns.nlevels > 1 and cols is None:
        # only standardize the first level
        new_df = df.copy()
        new_cols = [to_snake(c) for c in df.columns.levels[0]]
        new_df.columns = new_df.columns.set_levels(new_cols, level=0)
        return new_df

    if cols is None:
        cols = list(df.columns)

    return df.rename(columns={c: to_snake(c) for c in cols})
    
class WhoScored(BaseReader):
    def __init__(
        self, 
        leagues: Optional[Union[str, list[str]]] = None,
        seasons: Optional[Union[str, int, Iterable[Union[str, int]]]] = None,
    ):
        
        data_dir: Path = WHOSCORED_DATADIR
        """Initialize the WhoScored reader."""
        super().__init__(
            leagues=leagues,
            data_dir=data_dir,
        )

        self.seasons = seasons
    

    def read_leagues(self) -> pd.DataFrame:
        """Retrieve the selected leagues from the datasource.

        Returns
        -------
        pd.DataFrame
        """
        filepath = self.data_dir / "tiers.json"
        reader = filepath.open(mode="rb")
        data = json.load(reader)

        leagues = []
        for region in data:
            for league in region["tournaments"]:
                leagues.append(
                    {
                        "region_id": region["id"],
                        "region": region["name"],
                        "league_id": league["id"],
                        "league": league["name"],
                    }
                )

        return (
            pd.DataFrame(leagues)
            .assign(league=lambda x: x.region + " - " + x.league)
            .pipe(self._translate_league)
            .set_index("league")
            .loc[self._selected_leagues.keys()]
            .sort_index()
        )
    
    def read_seasons(self) -> pd.DataFrame:
        """Retrieve the selected seasons for the selected leagues.

        Returns
        -------
        pd.DataFrame
        """
        df_leagues = self.read_leagues()

        seasons = []
        for lkey, league in df_leagues.iterrows():
            filemask = "seasons/{}.html"
            filepath = self.data_dir / filemask.format(lkey)
            
            if not filepath.is_file():
                continue

            reader = filepath.open(mode="rb")

            # extract team links
            tree = html.parse(reader)
            for node in tree.xpath("//select[contains(@id,'seasons')]/option"):
                # extract team IDs from links
                season_url = node.get("value")
                season_id = _parse_url(season_url)["season_id"]
                seasons.append(
                    {
                        "league": lkey,
                        "season": self._season_code.parse(node.text),
                        "region_id": league.region_id,
                        "league_id": league.league_id,
                        "season_id": season_id,
                    }
                )
        
        idxs = [xx for xx in itertools.product(self.leagues, self.seasons) if xx in [(x['league'], x['season']) for x in seasons]]
        return (
            pd.DataFrame(seasons)
            .set_index(["league", "season"])
            .sort_index()
            .loc[idxs]
        )
    
    def read_season_stages(self, force_cache: bool = False) -> pd.DataFrame:
        """Retrieve the season stages for the selected leagues.

        Parameters
        ----------
        force_cache : bool
             By default no cached data is used for the current season.
             If True, will force the use of cached data anyway.

        Returns
        -------
        pd.DataFrame
        """
        df_seasons = self.read_seasons()
        filemask = "seasons/{}_{}.html"

        season_stages = []
        for (lkey, skey), season in df_seasons.iterrows():
            current_season = not self._is_complete(lkey, skey)

            # get season page
            filepath = self.data_dir / filemask.format(lkey, skey)

            if not filepath.is_file():
                continue

            reader = filepath.open(mode="rb")
            tree = html.parse(reader)

            # get default season stage
            fixtures_url = tree.xpath("//a[text()='Fixtures']/@href")[0]
            stage_id = _parse_url(fixtures_url)["stage_id"]
            season_stages.append(
                {
                    "league": lkey,
                    "season": skey,
                    "region_id": season.region_id,
                    "league_id": season.league_id,
                    "season_id": season.season_id,
                    "stage_id": stage_id,
                    "stage": None,
                }
            )

            # extract additional stages
            for node in tree.xpath("//select[contains(@id,'stages')]/option"):
                stage_url = node.get("value")
                stage_id = _parse_url(stage_url)["stage_id"]
                season_stages.append(
                    {
                        "league": lkey,
                        "season": skey,
                        "region_id": season.region_id,
                        "league_id": season.league_id,
                        "season_id": season.season_id,
                        "stage_id": stage_id,
                        "stage": node.text,
                    }
                )

        idxs = [xx for xx in itertools.product(self.leagues, self.seasons) if xx in [(x['league'], x['season']) for x in season_stages]]
        return (
            pd.DataFrame(season_stages)
            .drop_duplicates(subset=["league", "season", "stage_id"], keep="last")
            .set_index(["league", "season"])
            .sort_index()
            .loc[idxs]
        )
    
    def read_schedule(self, force_cache: bool = False) -> pd.DataFrame:
        """Retrieve the game schedule for the selected leagues and seasons.

        Parameters
        ----------
        force_cache : bool
             By default no cached data is used for the current season.
             If True, will force the use of cached data anyway.

        Returns
        -------
        pd.DataFrame
        """
        df_season_stages = self.read_season_stages(force_cache=force_cache)
        filemask_schedule = "matches/{}_{}_{}_{}.json"

        all_schedules = []
        for (lkey, skey), stage in df_season_stages.iterrows():
            current_season = not self._is_complete(lkey, skey)
            stage_id = stage["stage_id"]
            stage_name = stage["stage"]

            # get the calendar of the season stage
            if stage_name is not None:
                calendar_filepath = self.data_dir / f"matches/{lkey}_{skey}_{stage_id}.html"
                logger.info(
                    "Retrieving calendar for %s %s (%s)",
                    lkey,
                    skey,
                    stage_name,
                )
            else:
                calendar_filepath = self.data_dir / f"matches/{lkey}_{skey}.html"
                logger.info(
                    "Retrieving calendar for %s %s",
                    lkey,
                    skey,
                )

            if not calendar_filepath.is_file():
                continue

            calendar = calendar_filepath.open(mode="rb")
            mask = json.load(calendar)["mask"]

            # get the fixtures for each month
            it = [(year, month) for year in mask for month in mask[year]]
            for i, (year, month) in enumerate(it):
                filepath = self.data_dir / filemask_schedule.format(lkey, skey, stage_id, month)

                if stage_name is not None:
                    logger.info(
                        "[%s/%s] Retrieving fixtures for %s %s (%s)",
                        i + 1,
                        len(it),
                        lkey,
                        skey,
                        stage_name,
                    )
                else:
                    logger.info(
                        "[%s/%s] Retrieving fixtures for %s %s",
                        i + 1,
                        len(it),
                        lkey,
                        skey,
                    )

                if not filepath.is_file():
                    continue

                reader = filepath.open(mode="rb")
                data = json.load(reader)
                for tournament in data["tournaments"]:
                    df_schedule = pd.DataFrame(tournament["matches"])
                    df_schedule["league"] = lkey
                    df_schedule["season"] = skey
                    df_schedule["stage"] = stage_name
                    all_schedules.append(df_schedule)

        if len(all_schedules) == 0:
            return pd.DataFrame(index=["league", "season", "game"])

        # Construct the output dataframe
        return (
            pd.concat(all_schedules)
            .drop_duplicates(subset=["id"])
            .replace(
                {
                    "homeTeamName": TEAMNAME_REPLACEMENTS,
                    "awayTeamName": TEAMNAME_REPLACEMENTS,
                }
            )
            .rename(
                columns={
                    "homeTeamName": "home_team",
                    "awayTeamName": "away_team",
                    "id": "game_id",
                    "startTimeUtc": "date",
                }
            )
            .assign(date=lambda x: pd.to_datetime(x["date"]))
            .assign(game=lambda df: df.apply(make_game_id, axis=1))
            .pipe(standardize_colnames)
            .set_index(["league", "season", "game"])
            .sort_index()
        )
    
    def read_missing_players(
        self,
        match_id: Optional[Union[int, list[int]]] = None,
        force_cache: bool = False,
    ) -> pd.DataFrame:
        """Retrieve a list of injured and suspended players ahead of each game.

        Parameters
        ----------
        match_id : int or list of int, optional
            Retrieve the missing players for a specific game.
        force_cache : bool
            By default no cached data is used to scrapre the list of available
            games for the current season. If True, will force the use of
            cached data anyway.

        Raises
        ------
        ValueError
            If the given match_id could not be found in the selected seasons.

        Returns
        -------
        pd.DataFrame
        """
        filemask = "WhoScored/previews/{}_{}/{}.html"

        df_schedule = self.read_schedule(force_cache).reset_index()
        if match_id is not None:
            iterator = df_schedule[
                df_schedule.game_id.isin([match_id] if isinstance(match_id, int) else match_id)
            ]
            if len(iterator) == 0:
                raise ValueError("No games found with the given IDs in the selected seasons.")
        else:
            iterator = df_schedule.sample(frac=1)

        match_sheets = []
        for i, (_, game) in enumerate(iterator.iterrows()):
            filepath = DATA_DIR / filemask.format(game["league"], game["season"], game["game_id"])

            logger.info(
                "[%s/%s] Retrieving game with id=%s",
                i + 1,
                len(iterator),
                game["game_id"],
            )

            if not filepath.is_file():
                continue

            reader = filepath.open(mode="rb")

            # extract missing players
            tree = html.parse(reader)
            for node in tree.xpath("//div[@id='missing-players']/div[2]/table/tbody/tr"):
                # extract team IDs from links
                match_sheets.append(
                    {
                        "league": game["league"],
                        "season": game["season"],
                        "game": game["game"],
                        "game_id": game["game_id"],
                        "team": game["home_team"],
                        "player": node.xpath("./td[contains(@class,'pn')]/a")[0].text,
                        "player_id": int(
                            node.xpath("./td[contains(@class,'pn')]/a")[0]
                            .get("href")
                            .split("/")[2]
                        ),
                        "reason": node.xpath("./td[contains(@class,'reason')]/span")[0].get(
                            "title"
                        ),
                        "status": node.xpath("./td[contains(@class,'confirmed')]")[0].text,
                    }
                )
            for node in tree.xpath("//div[@id='missing-players']/div[3]/table/tbody/tr"):
                # extract team IDs from links
                match_sheets.append(
                    {
                        "league": game["league"],
                        "season": game["season"],
                        "game": game["game"],
                        "game_id": game["game_id"],
                        "team": game["away_team"],
                        "player": node.xpath("./td[contains(@class,'pn')]/a")[0].text,
                        "player_id": int(
                            node.xpath("./td[contains(@class,'pn')]/a")[0]
                            .get("href")
                            .split("/")[2]
                        ),
                        "reason": node.xpath("./td[contains(@class,'reason')]/span")[0].get(
                            "title"
                        ),
                        "status": node.xpath("./td[contains(@class,'confirmed')]")[0].text,
                    }
                )

        if len(match_sheets) == 0:
            return pd.DataFrame(index=["league", "season", "game", "team", "player"])

        return (
            pd.DataFrame(match_sheets)
            .set_index(["league", "season", "game", "team", "player"])
            .sort_index()
        )

    def read_events(  # noqa: C901
        self,
        match_id: Optional[Union[int, list[int]]] = None,
        force_cache: bool = False,
        live: bool = False,
        output_fmt: Optional[str] = "events",
        retry_missing: bool = True,
        on_error: Literal["raise", "skip"] = "raise",
    ) -> Optional[Union[pd.DataFrame, dict[int, list], "OptaLoader"]]:  # type: ignore  # noqa: F821
        """Retrieve the the event data for each game in the selected leagues and seasons.

        Parameters
        ----------
        match_id : int or list of int, optional
            Retrieve the event stream for a specific game.
        force_cache : bool
            By default no cached data is used to scrape the list of available
            games for the current season. If True, will force the use of
            cached data anyway.
        live : bool
            If True, will not return a cached copy of the event data. This is
            usefull to scrape live data.
        output_fmt : str, default: 'events'
            The output format of the returned data. Possible values are:
                - 'events' (default): Returns a dataframe with all events.
                - 'raw': Returns the original unformatted WhoScored JSON.
                - 'spadl': Returns a dataframe with the SPADL representation
                  of the original events.
                  See https://socceraction.readthedocs.io/en/latest/documentation/SPADL.html#spadl
                - 'atomic-spadl': Returns a dataframe with the Atomic-SPADL representation
                  of the original events.
                  See https://socceraction.readthedocs.io/en/latest/documentation/SPADL.html#atomic-spadl
                - 'loader': Returns a socceraction.data.opta.OptaLoader
                  instance, which can be used to retrieve the actual data.
                  See https://socceraction.readthedocs.io/en/latest/modules/generated/socceraction.data.opta.OptaLoader.html#socceraction.data.opta.OptaLoader
                - None: Doesn't return any data. This is useful to just cache
                  the data without storing the events in memory.
        retry_missing : bool
            If no events were found for a game in a previous attempt, will
            retry to scrape the events
        on_error : "raise" or "skip", default: "raise"
            Wheter to raise an exception or to skip the game if an error occurs.

        Raises
        ------
        ValueError
            If the given match_id could not be found in the selected seasons.
        ConnectionError
            If the match page could not be retrieved.
        ImportError
            If the requested output format is 'spadl', 'atomic-spadl' or
            'loader' but the socceraction package is not installed.

        Returns
        -------
        See the description of the ``output_fmt`` parameter.
        """
        output_fmt = output_fmt.lower() if output_fmt is not None else None
        if output_fmt in ["loader", "spadl", "atomic-spadl"]:
            if self.no_store:
                raise ValueError(
                    f"The '{output_fmt}' output format is not supported "
                    "when using the 'no_store' option."
                )
            try:
                from socceraction.atomic.spadl import convert_to_atomic
                from socceraction.data.opta import OptaLoader
                from socceraction.data.opta.loader import _eventtypesdf
                from socceraction.data.opta.parsers import WhoScoredParser
                from socceraction.spadl.opta import convert_to_actions

                if output_fmt == "loader":
                    import socceraction
                    from packaging import version

                    if version.parse(socceraction.__version__) < version.parse("1.2.3"):
                        raise ImportError(
                            "The 'loader' output format requires socceraction >= 1.2.3"
                        )
            except ImportError:
                raise ImportError(
                    "The socceraction package is required to use the 'spadl' "
                    "or 'atomic-spadl' output format. "
                    "Please install it with `pip install socceraction`."
                )
        
        filemask = "events/{}_{}/{}.json"

        df_schedule = self.read_schedule(force_cache).reset_index()
        if match_id is not None:
            iterator = df_schedule[
                df_schedule.game_id.isin([match_id] if isinstance(match_id, int) else match_id)
            ]
            if len(iterator) == 0:
                raise ValueError("No games found with the given IDs in the selected seasons.")
        else:
            iterator = df_schedule.sample(frac=1)

        events = {}
        player_names = {}
        team_names = {}
        for i, (_, game) in enumerate(iterator.iterrows()):
            # get league and season
            logger.info(
                "[%s/%s] Retrieving game with id=%s",
                i + 1,
                len(iterator),
                game["game_id"],
            )
            filepath = self.data_dir / filemask.format(
                game["league"], game["season"], game["game_id"]
            )

            try:
                if not filepath.is_file():
                    continue

                reader = filepath.open(mode="rb")
                reader_value = reader.read()
                if retry_missing and reader_value == b"null" or reader_value == b"":
                    reader = filepath.open(mode="rb")
            except ConnectionError as e:
                if on_error == "skip":
                    logger.warning("Error while scraping game %s: %s", game["game_id"], e)
                    continue
                raise
            reader.seek(0)
            json_data = json.load(reader)
            if json_data is not None:
                player_names.update(
                    {int(k): v for k, v in json_data["playerIdNameDictionary"].items()}
                )
                team_names.update(
                    {
                        int(json_data[side]["teamId"]): json_data[side]["name"]
                        for side in ["home", "away"]
                    }
                )
                if "events" in json_data:
                    game_events = json_data["events"]
                    if output_fmt == "events":
                        df_events = pd.DataFrame(game_events)
                        df_events["game"] = game["game"]
                        df_events["league"] = game["league"]
                        df_events["season"] = game["season"]
                        df_events["game_id"] = game["game_id"]
                        events[game["game_id"]] = df_events
                    elif output_fmt == "raw":
                        events[game["game_id"]] = game_events
                    elif output_fmt in ["spadl", "atomic-spadl"]:
                        parser = WhoScoredParser(
                            str(filepath),
                            competition_id=game["league"],
                            season_id=game["season"],
                            game_id=game["game_id"],
                        )
                        df_events = (
                            pd.DataFrame.from_dict(parser.extract_events(), orient="index")
                            .merge(_eventtypesdf, on="type_id", how="left")
                            .reset_index(drop=True)
                        )
                        df_actions = convert_to_actions(
                            df_events, home_team_id=int(json_data["home"]["teamId"])
                        )
                        if output_fmt == "spadl":
                            events[game["game_id"]] = df_actions
                        else:
                            events[game["game_id"]] = convert_to_atomic(df_actions)

            else:
                logger.warning("No events found for game %s", game["game_id"])

        if output_fmt is None:
            return None

        if output_fmt == "raw":
            return events

        if output_fmt == "loader":
            return OptaLoader(
                root=self.data_dir,
                parser="whoscored",
                feeds={
                    "whoscored": str(Path("events/{competition_id}_{season_id}/{game_id}.json"))
                },
            )

        if len(events) == 0:
            return pd.DataFrame(index=["league", "season", "game"])

        df = (
            pd.concat(events.values())
            .pipe(standardize_colnames)
            .assign(
                player=lambda x: x.player_id.replace(player_names),
                team=lambda x: x.team_id.replace(team_names).replace(TEAMNAME_REPLACEMENTS),
            )
        )

        if output_fmt == "events":
            df = df.set_index(["league", "season", "game"]).sort_index()
            # add missing columns
            for col, default in COLS_EVENTS.items():
                if col not in df.columns:
                    df[col] = default
            df["outcome_type"] = df["outcome_type"].apply(
                lambda x: x.get("displayName") if pd.notnull(x) else x
            )
            df["card_type"] = df["card_type"].apply(
                lambda x: x.get("displayName") if pd.notnull(x) else x
            )
            df["type"] = df["type"].apply(lambda x: x.get("displayName") if pd.notnull(x) else x)
            df["period"] = df["period"].apply(
                lambda x: x.get("displayName") if pd.notnull(x) else x
            )
            df = df[list(COLS_EVENTS.keys())]

        return df
    
    def _download_and_save(
        self,
        url: str,
        filepath: Optional[Path] = None,
        var: Optional[Union[str, Iterable[str]]] = None,
    ) -> IO[bytes]:
        pass
    

x = WhoScored().read_schedule()
print(x.game_id.values)
y = WhoScored().read_events(match_id=int(1375927), output_fmt="spadl", force_cache=True)
# WhoScored2()._translate_league(x)
# WhoScored()._selected_leagues()
# x.league.unique()
y


[1375927 1375929 1375930 ... 1746419 1746414 1746314]


,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,end_x,start_y,end_y,type_id,result_id,bodypart_id,action_id,player,team
0,1375927,1936039246,1,1.0,168,82699.0,52.395,57.015,34.204,34.272,0,1,0,0,Marco Stiepermann,Norwich
1,1375927,123520708,1,2.0,168,82917.0,58.275,72.660,32.504,15.436,0,1,0,1,Kenny McLean,Norwich
2,1375927,NaN,1,3.0,168,287504.0,72.660,75.810,15.436,15.844,21,1,0,2,Ben Godfrey,Norwich
3,1375927,1319101870,1,4.0,168,287504.0,75.810,93.870,15.844,29.308,0,1,0,3,Ben Godfrey,Norwich
4,1375927,NaN,1,5.5,168,22319.0,93.870,94.710,29.308,25.568,21,1,0,4,Tim Krul,Norwich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,1375927,1270876750,2,3044.0,168,287504.0,93.240,100.695,25.364,34.000,0,1,0,1525,Ben Godfrey,Norwich
1526,1375927,608934461,2,3047.0,168,22319.0,100.170,87.885,35.428,33.592,0,1,0,1526,Tim Krul,Norwich
1527,1375927,830330423,2,3049.0,168,82917.0,87.465,98.490,32.572,18.972,0,1,0,1527,Kenny McLean,Norwich
1528,1375927,2066242729,2,3051.0,168,287504.0,98.280,87.990,17.000,6.188,0,1,0,1528,Ben Godfrey,Norwich


In [22]:
from utils.soccerdata_utils import WhoScored

In [23]:
x = WhoScored().read_schedule()
print(x.game_id.values)
y = WhoScored().read_events(match_id=int(1375927), output_fmt="spadl", force_cache=True)
# WhoScored2()._translate_league(x)
# WhoScored()._selected_leagues()
# x.league.unique()
y


[1375927 1375929 1375930 ... 1746419 1746414 1746314]


,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,end_x,start_y,end_y,type_id,result_id,bodypart_id,action_id,player,team
0,1375927,1936039246,1,1.0,168,82699.0,52.395,57.015,34.204,34.272,0,1,0,0,Marco Stiepermann,Norwich
1,1375927,123520708,1,2.0,168,82917.0,58.275,72.660,32.504,15.436,0,1,0,1,Kenny McLean,Norwich
2,1375927,NaN,1,3.0,168,287504.0,72.660,75.810,15.436,15.844,21,1,0,2,Ben Godfrey,Norwich
3,1375927,1319101870,1,4.0,168,287504.0,75.810,93.870,15.844,29.308,0,1,0,3,Ben Godfrey,Norwich
4,1375927,NaN,1,5.5,168,22319.0,93.870,94.710,29.308,25.568,21,1,0,4,Tim Krul,Norwich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,1375927,1270876750,2,3044.0,168,287504.0,93.240,100.695,25.364,34.000,0,1,0,1525,Ben Godfrey,Norwich
1526,1375927,608934461,2,3047.0,168,22319.0,100.170,87.885,35.428,33.592,0,1,0,1526,Tim Krul,Norwich
1527,1375927,830330423,2,3049.0,168,82917.0,87.465,98.490,32.572,18.972,0,1,0,1527,Kenny McLean,Norwich
1528,1375927,2066242729,2,3051.0,168,287504.0,98.280,87.990,17.000,6.188,0,1,0,1528,Ben Godfrey,Norwich


In [89]:
s = []
s.append({'league': 1, 'season': 1.1})
s.append({'league': 2, 'season': 2.1})
[(x['league'], x['season']) for x in s]

[(1, 1.1), (2, 2.1)]

In [80]:
x = WhoScored()
[xx for xx in itertools.product(x.leagues, x.seasons) if xx in y.index]

WhoScored


[04/09/25 11:37:43] INFO     Saving cached data to /Users/vitaly/soccerdata/data/WhoScored           ]8;id=919746;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=884810;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_common.py#263\263]8;;\

                    INFO     No seasons provided. Will retrieve data for the last 5 seasons.         ]8;id=717087;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_common.py\_common.py]8;;\:]8;id=584341;file:///Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/python3.9/site-packages/soccerdata/_common.py#476\476]8;;\

                    WARNING  /Users/vitaly/Documents/Github_personal/Sport_radar_chart/.venv/lib/py ]8;id=677102;file:///Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/warnings.py\warnings.py]8;;\:]8;id=281556;file:///Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/warnings.py#109\109]8;;\
                             thon3.9/site-packages/soccerdata/_common.py:433: UserWarning: The                     
                             leagues have different default season codes. Using multi-year season                  
                             codes.                                                                                
                                                                                                                   
                                                                                                                   

[('ENG-Premier League', '2425'),
 ('ENG-Premier League', '2324'),
 ('ENG-Premier League', '2223'),
 ('ENG-Premier League', '2122'),
 ('ENG-Premier League', '2021'),
 ('ENG-Premier League', '1920'),
 ('ESP-La Liga', '2425'),
 ('ESP-La Liga', '2324'),
 ('ESP-La Liga', '2223'),
 ('ESP-La Liga', '2122'),
 ('ESP-La Liga', '2021'),
 ('ESP-La Liga', '1920'),
 ('ITA-Serie A', '2425'),
 ('ITA-Serie A', '2324'),
 ('ITA-Serie A', '2223'),
 ('ITA-Serie A', '2122'),
 ('ITA-Serie A', '2021'),
 ('ITA-Serie A', '1920'),
 ('GER-Bundesliga', '2425'),
 ('GER-Bundesliga', '2324'),
 ('GER-Bundesliga', '2223'),
 ('GER-Bundesliga', '2122'),
 ('GER-Bundesliga', '2021'),
 ('GER-Bundesliga', '1920'),
 ('FRA-Ligue 1', '2425'),
 ('FRA-Ligue 1', '2324'),
 ('FRA-Ligue 1', '2223'),
 ('FRA-Ligue 1', '2122'),
 ('FRA-Ligue 1', '2021'),
 ('FRA-Ligue 1', '1920'),
 ('INT-World Cup', '2223'),
 ('INT-European Championship', '2425'),
 ('INT-European Championship', '2021')]

In [54]:
lkey = 'ENG-Premier League'
filepath = Path('/Users/vitaly/soccerdata/data/WhoScored/seasons/ENG-Premier League.html')

reader = filepath.open(mode="rb")
tree = html.parse(reader)
seasons = []
for node in tree.xpath("//select[contains(@id,'seasons')]/option"):
    # extract team IDs from links
    season_url = node.get("value")
    season_id = _parse_url(season_url)["season_id"]
    seasons.append(
        {
            "league": lkey,
            # "season": self._season_code.parse(node.text),
            # "region_id": league.region_id,
            # "league_id": league.league_id,
            "season_id": season_id,
        }
    )
seasons

[{'league': 'ENG-Premier League', 'season_id': '10316'},
 {'league': 'ENG-Premier League', 'season_id': '9618'},
 {'league': 'ENG-Premier League', 'season_id': '9075'},
 {'league': 'ENG-Premier League', 'season_id': '8618'},
 {'league': 'ENG-Premier League', 'season_id': '8228'},
 {'league': 'ENG-Premier League', 'season_id': '7811'},
 {'league': 'ENG-Premier League', 'season_id': '7361'},
 {'league': 'ENG-Premier League', 'season_id': '6829'},
 {'league': 'ENG-Premier League', 'season_id': '6335'},
 {'league': 'ENG-Premier League', 'season_id': '5826'},
 {'league': 'ENG-Premier League', 'season_id': '4311'},
 {'league': 'ENG-Premier League', 'season_id': '3853'},
 {'league': 'ENG-Premier League', 'season_id': '3389'},
 {'league': 'ENG-Premier League', 'season_id': '2935'},
 {'league': 'ENG-Premier League', 'season_id': '2458'},
 {'league': 'ENG-Premier League', 'season_id': '1849'},
 {'league': 'ENG-Premier League', 'season_id': '1539'},
 {'league': 'ENG-Premier League', 'season_id': 

In [37]:
!head /Users/vitaly/soccerdata/data/WhoScored/seasons/ENG-Premier League.html

head: /Users/vitaly/soccerdata/data/WhoScored/seasons/ENG-Premier: No such file or directory
head: League.html: No such file or directory


In [11]:
WhoScored()._all_leagues_dict

{'ENG-Premier League': 'England - Premier League',
 'ESP-La Liga': 'Spain - LaLiga',
 'ITA-Serie A': 'Italy - Serie A',
 'GER-Bundesliga': 'Germany - Bundesliga',
 'FRA-Ligue 1': 'France - Ligue 1',
 'INT-World Cup': 'International - FIFA World Cup',
 'INT-European Championship': 'International - European Championship',
 "INT-Women's World Cup": "International - FIFA Women's World Cup"}

In [ ]:
"""Scraper for http://whoscored.com."""

import itertools
import json
import re
import time
from collections.abc import Iterable
from pathlib import Path
from typing import Callable, Literal, Optional, Union

import numpy as np
import pandas as pd

from ._common import BaseSeleniumReader, make_game_id, standardize_colnames
from ._config import DATA_DIR, NOCACHE, NOSTORE, TEAMNAME_REPLACEMENTS, logger

WHOSCORED_DATADIR = DATA_DIR / "WhoScored"

COLS_EVENTS = {
    # The ID of the game
    "game_id": np.nan,
    # 'PreMatch', 'FirstHalf', 'SecondHalf', 'PostGame'
    "period": np.nan,
    # Integer indicating the minute of the event, ignoring stoppage time
    "minute": -1,
    # Integer indicating the second of the event, ignoring stoppage time
    "second": -1,
    # Integer indicating the minute of the event, taking into account stoppage time
    "expanded_minute": -1,
    # String describing the event type (e.g. 'Goal', 'Yellow Card', etc.)
    "type": np.nan,
    # String describing the event outcome ('Succesful' or 'Unsuccessful')
    "outcome_type": np.nan,
    # The ID of the team that the event is associated with
    "team_id": np.nan,
    # The name of the team that the event is associated with
    "team": np.nan,
    # The ID of the player that the event is associated with
    "player_id": np.nan,
    # The name of the player that the event is associated with
    "player": np.nan,
    # Coordinates of the event's location
    "x": np.nan,
    "y": np.nan,
    "end_x": np.nan,
    "end_y": np.nan,
    # Coordinates of a shot's location
    "goal_mouth_y": np.nan,
    "goal_mouth_z": np.nan,
    # The coordinates where the ball was blocked
    "blocked_x": np.nan,
    "blocked_y": np.nan,
    # List of dicts with event qualifiers
    "qualifiers": [],
    # Some boolean flags
    "is_touch": False,
    "is_shot": False,
    "is_goal": False,
    # 'Yellow', 'Red', 'SecondYellow'
    "card_type": np.nan,
    # The ID of an associated event
    "related_event_id": np.nan,
    # The ID of a secondary player that the event is associated with
    "related_player_id": np.nan,
}


class WhoScored():
    """Provides pd.DataFrames from data available at http://whoscored.com.

    Data will be downloaded as necessary and cached locally in
    ``~/soccerdata/data/WhoScored``.

    Parameters
    ----------
    leagues : string or iterable, optional
        IDs of Leagues to include.
    seasons : string, int or list, optional
        Seasons to include. Supports multiple formats.
        Examples: '16-17'; 2016; '2016-17'; [14, 15, 16]
    proxy : 'tor' or dict or list(dict) or callable, optional
        Use a proxy to hide your IP address. Valid options are:
            - "tor": Uses the Tor network. Tor should be running in
              the background on port 9050.
            - dict: A dictionary with the proxy to use. The dict should be
              a mapping of supported protocols to proxy addresses. For example::

                  {
                      'http': 'http://10.10.1.10:3128',
                      'https': 'http://10.10.1.10:1080',
                  }

            - list(dict): A list of proxies to choose from. A different proxy will
              be selected from this list after failed requests, allowing rotating
              proxies.
            - callable: A function that returns a valid proxy. This function will
              be called after failed requests, allowing rotating proxies.
    no_cache : bool
        If True, will not use cached data.
    no_store : bool
        If True, will not store downloaded data.
    data_dir : Path
        Path to directory where data will be cached.
    path_to_browser : Path, optional
        Path to the Chrome executable.
    headless : bool, default: True
        If True, will run Chrome in headless mode. Setting this to False might
        help to avoid getting blocked. Only supported for Selenium <4.13.
    """

    def __init__(
        self,
        leagues: Optional[Union[str, list[str]]] = None,
        seasons: Optional[Union[str, int, Iterable[Union[str, int]]]] = None,
        proxy: Optional[
            Union[str, dict[str, str], list[dict[str, str]], Callable[[], dict[str, str]]]
        ] = None,
        no_cache: bool = NOCACHE,
        no_store: bool = NOSTORE,
        data_dir: Path = WHOSCORED_DATADIR,
        path_to_browser: Optional[Path] = None,
        headless: bool = False,
    ):
        """Initialize the WhoScored reader."""
        super().__init__(
            leagues=leagues,
            proxy=proxy,
            no_cache=no_cache,
            no_store=no_store,
            data_dir=data_dir,
            path_to_browser=path_to_browser,
            headless=headless,
        )
        self.seasons = seasons  # type: ignore
        self.rate_limit = 5
        self.max_delay = 5
        if not self.no_store:
            (self.data_dir / "seasons").mkdir(parents=True, exist_ok=True)
            (self.data_dir / "matches").mkdir(parents=True, exist_ok=True)
            (self.data_dir / "previews").mkdir(parents=True, exist_ok=True)
            (self.data_dir / "events").mkdir(parents=True, exist_ok=True)

    def read_leagues(self) -> pd.DataFrame:
        """Retrieve the selected leagues from the datasource.

        Returns
        -------
        pd.DataFrame
        """
        filepath = self.data_dir / "tiers.json"
        reader = filepath.open(mode="rb")
        data = json.load(reader)

        leagues = []
        for region in data:
            for league in region["tournaments"]:
                leagues.append(
                    {
                        "region_id": region["id"],
                        "region": region["name"],
                        "league_id": league["id"],
                        "league": league["name"],
                    }
                )

        return (
            pd.DataFrame(leagues)
            .assign(league=lambda x: x.region + " - " + x.league)
            .pipe(self._translate_league)
            .set_index("league")
            .loc[self._selected_leagues.keys()]
            .sort_index()
        )

    def read_seasons(self) -> pd.DataFrame:
        """Retrieve the selected seasons for the selected leagues.

        Returns
        -------
        pd.DataFrame
        """
        df_leagues = self.read_leagues()

        seasons = []
        for lkey, league in df_leagues.iterrows():
            url = (
                WHOSCORED_URL
                + f"/Regions/{league['region_id']}"
                + f"/Tournaments/{league['league_id']}"
            )
            filemask = "seasons/{}.html"
            filepath = self.data_dir / filemask.format(lkey)
            reader = self.get(url, filepath, var=None)

            # extract team links
            tree = html.parse(reader)
            for node in tree.xpath("//select[contains(@id,'seasons')]/option"):
                # extract team IDs from links
                season_url = node.get("value")
                season_id = _parse_url(season_url)["season_id"]
                seasons.append(
                    {
                        "league": lkey,
                        "season": self._season_code.parse(node.text),
                        "region_id": league.region_id,
                        "league_id": league.league_id,
                        "season_id": season_id,
                    }
                )

        return (
            pd.DataFrame(seasons)
            .set_index(["league", "season"])
            .sort_index()
            .loc[itertools.product(self.leagues, self.seasons)]
        )

    def read_season_stages(self, force_cache: bool = False) -> pd.DataFrame:
        """Retrieve the season stages for the selected leagues.

        Parameters
        ----------
        force_cache : bool
             By default no cached data is used for the current season.
             If True, will force the use of cached data anyway.

        Returns
        -------
        pd.DataFrame
        """
        df_seasons = self.read_seasons()
        filemask = "seasons/{}_{}.html"

        season_stages = []
        for (lkey, skey), season in df_seasons.iterrows():
            current_season = not self._is_complete(lkey, skey)

            # get season page
            url = (
                WHOSCORED_URL
                + f"/Regions/{season['region_id']}"
                + f"/Tournaments/{season['league_id']}"
                + f"/Seasons/{season['season_id']}"
            )
            filepath = self.data_dir / filemask.format(lkey, skey)
            reader = self.get(url, filepath, var=None, no_cache=current_season and not force_cache)
            tree = html.parse(reader)

            # get default season stage
            fixtures_url = tree.xpath("//a[text()='Fixtures']/@href")[0]
            stage_id = _parse_url(fixtures_url)["stage_id"]
            season_stages.append(
                {
                    "league": lkey,
                    "season": skey,
                    "region_id": season.region_id,
                    "league_id": season.league_id,
                    "season_id": season.season_id,
                    "stage_id": stage_id,
                    "stage": None,
                }
            )

            # extract additional stages
            for node in tree.xpath("//select[contains(@id,'stages')]/option"):
                stage_url = node.get("value")
                stage_id = _parse_url(stage_url)["stage_id"]
                season_stages.append(
                    {
                        "league": lkey,
                        "season": skey,
                        "region_id": season.region_id,
                        "league_id": season.league_id,
                        "season_id": season.season_id,
                        "stage_id": stage_id,
                        "stage": node.text,
                    }
                )

        return (
            pd.DataFrame(season_stages)
            .drop_duplicates(subset=["league", "season", "stage_id"], keep="last")
            .set_index(["league", "season"])
            .sort_index()
            .loc[itertools.product(self.leagues, self.seasons)]
        )

    def read_schedule(self, force_cache: bool = False) -> pd.DataFrame:
        """Retrieve the game schedule for the selected leagues and seasons.

        Parameters
        ----------
        force_cache : bool
             By default no cached data is used for the current season.
             If True, will force the use of cached data anyway.

        Returns
        -------
        pd.DataFrame
        """
        df_season_stages = self.read_season_stages(force_cache=force_cache)
        filemask_schedule = "matches/{}_{}_{}_{}.json"

        all_schedules = []
        for (lkey, skey), stage in df_season_stages.iterrows():
            current_season = not self._is_complete(lkey, skey)
            stage_id = stage["stage_id"]
            stage_name = stage["stage"]

            # get the calendar of the season stage
            season_stage_url = (
                WHOSCORED_URL
                + f"/Regions/{stage['region_id']}"
                + f"/Tournaments/{stage['league_id']}"
                + f"/Seasons/{stage['season_id']}"
                + f"/Stages/{stage['stage_id']}"
            )
            if stage_name is not None:
                calendar_filepath = self.data_dir / f"matches/{lkey}_{skey}_{stage_id}.html"
                logger.info(
                    "Retrieving calendar for %s %s (%s)",
                    lkey,
                    skey,
                    stage_name,
                )
            else:
                calendar_filepath = self.data_dir / f"matches/{lkey}_{skey}.html"
                logger.info(
                    "Retrieving calendar for %s %s",
                    lkey,
                    skey,
                )
            calendar = self.get(
                season_stage_url,
                calendar_filepath,
                var="wsCalendar",
                no_cache=current_season and not force_cache,
            )
            mask = json.load(calendar)["mask"]

            # get the fixtures for each month
            it = [(year, month) for year in mask for month in mask[year]]
            for i, (year, month) in enumerate(it):
                filepath = self.data_dir / filemask_schedule.format(lkey, skey, stage_id, month)
                url = WHOSCORED_URL + f"/tournaments/{stage_id}/data/?d={year}{(int(month)+1):02d}"

                if stage_name is not None:
                    logger.info(
                        "[%s/%s] Retrieving fixtures for %s %s (%s)",
                        i + 1,
                        len(it),
                        lkey,
                        skey,
                        stage_name,
                    )
                else:
                    logger.info(
                        "[%s/%s] Retrieving fixtures for %s %s",
                        i + 1,
                        len(it),
                        lkey,
                        skey,
                    )

                reader = self.get(
                    url, filepath, var=None, no_cache=current_season and not force_cache
                )
                data = json.load(reader)
                for tournament in data["tournaments"]:
                    df_schedule = pd.DataFrame(tournament["matches"])
                    df_schedule["league"] = lkey
                    df_schedule["season"] = skey
                    df_schedule["stage"] = stage_name
                    all_schedules.append(df_schedule)

        if len(all_schedules) == 0:
            return pd.DataFrame(index=["league", "season", "game"])

        # Construct the output dataframe
        return (
            pd.concat(all_schedules)
            .drop_duplicates(subset=["id"])
            .replace(
                {
                    "homeTeamName": TEAMNAME_REPLACEMENTS,
                    "awayTeamName": TEAMNAME_REPLACEMENTS,
                }
            )
            .rename(
                columns={
                    "homeTeamName": "home_team",
                    "awayTeamName": "away_team",
                    "id": "game_id",
                    "startTimeUtc": "date",
                }
            )
            .assign(date=lambda x: pd.to_datetime(x["date"]))
            .assign(game=lambda df: df.apply(make_game_id, axis=1))
            .pipe(standardize_colnames)
            .set_index(["league", "season", "game"])
            .sort_index()
        )

    def _read_game_info(self, game_id: int) -> dict:
        """Return game info available in the header."""
        urlmask = WHOSCORED_URL + "/Matches/{}"
        url = urlmask.format(game_id)
        data = {}
        self._driver.get(url)
        # league and season
        breadcrumb = self._driver.find_elements(
            By.XPATH,
            "//div[@id='breadcrumb-nav']/*[not(contains(@class, 'separator'))]",
        )
        country = breadcrumb[0].text
        league, season = breadcrumb[1].text.split(" - ")
        data["league"] = {v: k for k, v in self._all_leagues().items()}[f"{country} - {league}"]
        data["season"] = self._season_code.parse(season)
        # match header
        match_header = self._driver.find_element(By.XPATH, "//div[@id='match-header']")
        score_info = match_header.find_element(By.XPATH, ".//div[@class='teams-score-info']")
        data["home_team"] = score_info.find_element(
            By.XPATH, "./span[contains(@class,'home team')]"
        ).text
        data["result"] = score_info.find_element(
            By.XPATH, "./span[contains(@class,'result')]"
        ).text
        data["away_team"] = score_info.find_element(
            By.XPATH, "./span[contains(@class,'away team')]"
        ).text
        info_blocks = match_header.find_elements(By.XPATH, ".//div[@class='info-block cleared']")
        for block in info_blocks:
            for desc_list in block.find_elements(By.TAG_NAME, "dl"):
                for desc_def in desc_list.find_elements(By.TAG_NAME, "dt"):
                    desc_val = desc_def.find_element(By.XPATH, "./following-sibling::dd")
                    data[desc_def.text] = desc_val.text

        return data

    def read_missing_players(
        self,
        match_id: Optional[Union[int, list[int]]] = None,
        force_cache: bool = False,
    ) -> pd.DataFrame:
        """Retrieve a list of injured and suspended players ahead of each game.

        Parameters
        ----------
        match_id : int or list of int, optional
            Retrieve the missing players for a specific game.
        force_cache : bool
            By default no cached data is used to scrapre the list of available
            games for the current season. If True, will force the use of
            cached data anyway.

        Raises
        ------
        ValueError
            If the given match_id could not be found in the selected seasons.

        Returns
        -------
        pd.DataFrame
        """
        urlmask = WHOSCORED_URL + "/Matches/{}/Preview"
        filemask = "WhoScored/previews/{}_{}/{}.html"

        df_schedule = self.read_schedule(force_cache).reset_index()
        if match_id is not None:
            iterator = df_schedule[
                df_schedule.game_id.isin([match_id] if isinstance(match_id, int) else match_id)
            ]
            if len(iterator) == 0:
                raise ValueError("No games found with the given IDs in the selected seasons.")
        else:
            iterator = df_schedule.sample(frac=1)

        match_sheets = []
        for i, (_, game) in enumerate(iterator.iterrows()):
            url = urlmask.format(game.game_id)
            filepath = DATA_DIR / filemask.format(game["league"], game["season"], game["game_id"])

            logger.info(
                "[%s/%s] Retrieving game with id=%s",
                i + 1,
                len(iterator),
                game["game_id"],
            )
            reader = self.get(url, filepath, var=None)

            # extract missing players
            tree = html.parse(reader)
            for node in tree.xpath("//div[@id='missing-players']/div[2]/table/tbody/tr"):
                # extract team IDs from links
                match_sheets.append(
                    {
                        "league": game["league"],
                        "season": game["season"],
                        "game": game["game"],
                        "game_id": game["game_id"],
                        "team": game["home_team"],
                        "player": node.xpath("./td[contains(@class,'pn')]/a")[0].text,
                        "player_id": int(
                            node.xpath("./td[contains(@class,'pn')]/a")[0]
                            .get("href")
                            .split("/")[2]
                        ),
                        "reason": node.xpath("./td[contains(@class,'reason')]/span")[0].get(
                            "title"
                        ),
                        "status": node.xpath("./td[contains(@class,'confirmed')]")[0].text,
                    }
                )
            for node in tree.xpath("//div[@id='missing-players']/div[3]/table/tbody/tr"):
                # extract team IDs from links
                match_sheets.append(
                    {
                        "league": game["league"],
                        "season": game["season"],
                        "game": game["game"],
                        "game_id": game["game_id"],
                        "team": game["away_team"],
                        "player": node.xpath("./td[contains(@class,'pn')]/a")[0].text,
                        "player_id": int(
                            node.xpath("./td[contains(@class,'pn')]/a")[0]
                            .get("href")
                            .split("/")[2]
                        ),
                        "reason": node.xpath("./td[contains(@class,'reason')]/span")[0].get(
                            "title"
                        ),
                        "status": node.xpath("./td[contains(@class,'confirmed')]")[0].text,
                    }
                )

        if len(match_sheets) == 0:
            return pd.DataFrame(index=["league", "season", "game", "team", "player"])

        return (
            pd.DataFrame(match_sheets)
            .set_index(["league", "season", "game", "team", "player"])
            .sort_index()
        )

    def read_events(  # noqa: C901
        self,
        match_id: Optional[Union[int, list[int]]] = None,
        force_cache: bool = False,
        live: bool = False,
        output_fmt: Optional[str] = "events",
        retry_missing: bool = True,
        on_error: Literal["raise", "skip"] = "raise",
    ) -> Optional[Union[pd.DataFrame, dict[int, list], "OptaLoader"]]:  # type: ignore  # noqa: F821
        """Retrieve the the event data for each game in the selected leagues and seasons.

        Parameters
        ----------
        match_id : int or list of int, optional
            Retrieve the event stream for a specific game.
        force_cache : bool
            By default no cached data is used to scrape the list of available
            games for the current season. If True, will force the use of
            cached data anyway.
        live : bool
            If True, will not return a cached copy of the event data. This is
            usefull to scrape live data.
        output_fmt : str, default: 'events'
            The output format of the returned data. Possible values are:
                - 'events' (default): Returns a dataframe with all events.
                - 'raw': Returns the original unformatted WhoScored JSON.
                - 'spadl': Returns a dataframe with the SPADL representation
                  of the original events.
                  See https://socceraction.readthedocs.io/en/latest/documentation/SPADL.html#spadl
                - 'atomic-spadl': Returns a dataframe with the Atomic-SPADL representation
                  of the original events.
                  See https://socceraction.readthedocs.io/en/latest/documentation/SPADL.html#atomic-spadl
                - 'loader': Returns a socceraction.data.opta.OptaLoader
                  instance, which can be used to retrieve the actual data.
                  See https://socceraction.readthedocs.io/en/latest/modules/generated/socceraction.data.opta.OptaLoader.html#socceraction.data.opta.OptaLoader
                - None: Doesn't return any data. This is useful to just cache
                  the data without storing the events in memory.
        retry_missing : bool
            If no events were found for a game in a previous attempt, will
            retry to scrape the events
        on_error : "raise" or "skip", default: "raise"
            Wheter to raise an exception or to skip the game if an error occurs.

        Raises
        ------
        ValueError
            If the given match_id could not be found in the selected seasons.
        ConnectionError
            If the match page could not be retrieved.
        ImportError
            If the requested output format is 'spadl', 'atomic-spadl' or
            'loader' but the socceraction package is not installed.

        Returns
        -------
        See the description of the ``output_fmt`` parameter.
        """
        output_fmt = output_fmt.lower() if output_fmt is not None else None
        if output_fmt in ["loader", "spadl", "atomic-spadl"]:
            if self.no_store:
                raise ValueError(
                    f"The '{output_fmt}' output format is not supported "
                    "when using the 'no_store' option."
                )
            try:
                from socceraction.atomic.spadl import convert_to_atomic
                from socceraction.data.opta import OptaLoader
                from socceraction.data.opta.loader import _eventtypesdf
                from socceraction.data.opta.parsers import WhoScoredParser
                from socceraction.spadl.opta import convert_to_actions

                if output_fmt == "loader":
                    import socceraction
                    from packaging import version

                    if version.parse(socceraction.__version__) < version.parse("1.2.3"):
                        raise ImportError(
                            "The 'loader' output format requires socceraction >= 1.2.3"
                        )
            except ImportError:
                raise ImportError(
                    "The socceraction package is required to use the 'spadl' "
                    "or 'atomic-spadl' output format. "
                    "Please install it with `pip install socceraction`."
                )
        urlmask = WHOSCORED_URL + "/Matches/{}/Live"
        filemask = "events/{}_{}/{}.json"

        df_schedule = self.read_schedule(force_cache).reset_index()
        if match_id is not None:
            iterator = df_schedule[
                df_schedule.game_id.isin([match_id] if isinstance(match_id, int) else match_id)
            ]
            if len(iterator) == 0:
                raise ValueError("No games found with the given IDs in the selected seasons.")
        else:
            iterator = df_schedule.sample(frac=1)

        events = {}
        player_names = {}
        team_names = {}
        for i, (_, game) in enumerate(iterator.iterrows()):
            url = urlmask.format(game["game_id"])
            # get league and season
            logger.info(
                "[%s/%s] Retrieving game with id=%s",
                i + 1,
                len(iterator),
                game["game_id"],
            )
            filepath = self.data_dir / filemask.format(
                game["league"], game["season"], game["game_id"]
            )

            try:
                reader = self.get(
                    url,
                    filepath,
                    var="require.config.params['args'].matchCentreData",
                    no_cache=live,
                )
                reader_value = reader.read()
                if retry_missing and reader_value == b"null" or reader_value == b"":
                    reader = self.get(
                        url,
                        filepath,
                        var="require.config.params['args'].matchCentreData",
                        no_cache=True,
                    )
            except ConnectionError as e:
                if on_error == "skip":
                    logger.warning("Error while scraping game %s: %s", game["game_id"], e)
                    continue
                raise
            reader.seek(0)
            json_data = json.load(reader)
            if json_data is not None:
                player_names.update(
                    {int(k): v for k, v in json_data["playerIdNameDictionary"].items()}
                )
                team_names.update(
                    {
                        int(json_data[side]["teamId"]): json_data[side]["name"]
                        for side in ["home", "away"]
                    }
                )
                if "events" in json_data:
                    game_events = json_data["events"]
                    if output_fmt == "events":
                        df_events = pd.DataFrame(game_events)
                        df_events["game"] = game["game"]
                        df_events["league"] = game["league"]
                        df_events["season"] = game["season"]
                        df_events["game_id"] = game["game_id"]
                        events[game["game_id"]] = df_events
                    elif output_fmt == "raw":
                        events[game["game_id"]] = game_events
                    elif output_fmt in ["spadl", "atomic-spadl"]:
                        parser = WhoScoredParser(
                            str(filepath),
                            competition_id=game["league"],
                            season_id=game["season"],
                            game_id=game["game_id"],
                        )
                        df_events = (
                            pd.DataFrame.from_dict(parser.extract_events(), orient="index")
                            .merge(_eventtypesdf, on="type_id", how="left")
                            .reset_index(drop=True)
                        )
                        df_actions = convert_to_actions(
                            df_events, home_team_id=int(json_data["home"]["teamId"])
                        )
                        if output_fmt == "spadl":
                            events[game["game_id"]] = df_actions
                        else:
                            events[game["game_id"]] = convert_to_atomic(df_actions)

            else:
                logger.warning("No events found for game %s", game["game_id"])

        if output_fmt is None:
            return None

        if output_fmt == "raw":
            return events

        if output_fmt == "loader":
            return OptaLoader(
                root=self.data_dir,
                parser="whoscored",
                feeds={
                    "whoscored": str(Path("events/{competition_id}_{season_id}/{game_id}.json"))
                },
            )

        if len(events) == 0:
            return pd.DataFrame(index=["league", "season", "game"])

        df = (
            pd.concat(events.values())
            .pipe(standardize_colnames)
            .assign(
                player=lambda x: x.player_id.replace(player_names),
                team=lambda x: x.team_id.replace(team_names).replace(TEAMNAME_REPLACEMENTS),
            )
        )

        if output_fmt == "events":
            df = df.set_index(["league", "season", "game"]).sort_index()
            # add missing columns
            for col, default in COLS_EVENTS.items():
                if col not in df.columns:
                    df[col] = default
            df["outcome_type"] = df["outcome_type"].apply(
                lambda x: x.get("displayName") if pd.notnull(x) else x
            )
            df["card_type"] = df["card_type"].apply(
                lambda x: x.get("displayName") if pd.notnull(x) else x
            )
            df["type"] = df["type"].apply(lambda x: x.get("displayName") if pd.notnull(x) else x)
            df["period"] = df["period"].apply(
                lambda x: x.get("displayName") if pd.notnull(x) else x
            )
            df = df[list(COLS_EVENTS.keys())]

        return df

    def _handle_banner(self) -> None:
        try:
            # self._driver.get(WHOSCORED_URL)
            time.sleep(2)
            self._driver.find_element(By.XPATH, "//button[./span[text()='AGREE']]").click()
            time.sleep(2)
        except NoSuchElementException:
            # with open("/tmp/error.html", "w") as f:
            # f.write(self._driver.page_source)
            raise ElementClickInterceptedException()
